In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split

# Load DST Index Data
def load_dst_data(file_path):
    data = pd.read_csv(file_path)
    data["ds"] = pd.to_datetime(data["ds"])
    data.set_index("ds", inplace=True)
    return data

# Load DST dataset (Fine-tuning data)
dst_data = load_dst_data("dst_data_1975_2025.csv")

# Prepare Data for LSTM Model
dst_values = dst_data["y"].values

print(dst_values.shape, dst_values)

(438841,) [ -7.  -9. -10. ...   5.   6.  13.]


In [2]:
# Define the sliding window data preparation
class TimeSeriesDataset(Dataset):
    def __init__(self, data, window_size):
        self.data = data
        self.window_size = window_size

    def __len__(self):
        return len(self.data) - self.window_size

    def __getitem__(self, idx):
        x = self.data[idx: idx + self.window_size]
        y = self.data[idx + self.window_size]  # assuming DST is the first column
        # return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)
        return torch.tensor(x, dtype=torch.float32).unsqueeze(-1), torch.tensor(y, dtype=torch.float32)

In [3]:
def sliding_window_split(data, train_window_size, val_window_size):
    total_size = len(data)
    splits = []
    for start in range(0, total_size - train_window_size - val_window_size, val_window_size):
        train_indices = np.arange(start, start + train_window_size)
        val_indices = np.arange(start + train_window_size, start + train_window_size + val_window_size)
        splits.append((train_indices, val_indices))
    return splits

def create_dataloader_from_indices(data, indices, window_size, batch_size=64):
    subset_data = data[indices]
    dataset = TimeSeriesDataset(subset_data, window_size)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [4]:
class BidirectionalLSTM(nn.Module):
    def __init__(self):
        super(BidirectionalLSTM, self).__init__()
        self.lstm1 = nn.LSTM(1, 512, batch_first=True, bidirectional=True)
        self.dropout1 = nn.Dropout(0.2)
        self.lstm2 = nn.LSTM(512 * 2, 256, batch_first=True, bidirectional=True)
        self.dropout2 = nn.Dropout(0.2)
        self.lstm3 = nn.LSTM(256 * 2, 128, batch_first=True)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.dropout1(x)
        x, _ = self.lstm2(x)
        x = self.dropout2(x)
        x, _ = self.lstm3(x)
        x = self.fc1(x[:, -1, :])  # Get the last hidden state
        x = self.fc2(x)
        return x

In [5]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = np.inf
        self.early_stop_count = 0

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.early_stop_count = 0
        else:
            self.early_stop_count += 1
        return self.early_stop_count >= self.patience


In [6]:
def train_with_sliding_window(data, window_size, train_window_size, val_window_size, lr=1e-3, batch_size=64, num_epochs=50):
    print("OK1")
    splits = sliding_window_split(data, train_window_size, val_window_size)
    print("OK2")
    # TensorBoard writer
    writer = SummaryWriter(log_dir=f"runs/BiLSTM_window{window_size}_lr{lr}_batch{batch_size}")
    print("OK3")
    val_losses = []
    mae_scores = []
    print("OK4")
    for fold, (train_indices, val_indices) in enumerate(splits):
        print(f"Training fold {fold + 1}/{len(splits)}...")

        train_loader = create_dataloader_from_indices(data, train_indices, window_size, batch_size)
        val_loader = create_dataloader_from_indices(data, val_indices, window_size, batch_size)
        print("OK5")
        model = BidirectionalLSTM()
        print("OK6")

        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)
        # early_stopping = EarlyStopping(patience=5, delta=0.001)
        print("OK7")

        # Training loop
        for epoch in range(num_epochs):
            print("OK8")
            model.train()
            train_loss = 0
            print("OK9")
            for i, (x_batch, y_batch) in enumerate(train_loader):
                if i % 1000 == 1:
                    print(i)
                optimizer.zero_grad()
                output = model(x_batch)
                output = output.reshape(-1)
                loss = criterion(output, y_batch)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
            print(train_loss)

            # Validation loop
            model.eval()
            val_loss = 0
            all_preds = []
            all_true = []
            with torch.no_grad():
                for x_batch, y_batch in val_loader:
                    output = model(x_batch)
                    output = output.reshape(-1)
                    loss = criterion(output, y_batch)
                    val_loss += loss.item()
                    all_preds.append(output.cpu().numpy())
                    all_true.append(y_batch.cpu().numpy())

            train_loss /= len(train_loader)
            val_loss /= len(val_loader)

            # MAE Calculation
            all_preds = np.concatenate(all_preds, axis=0)
            all_true = np.concatenate(all_true, axis=0)
            mae = mean_absolute_error(all_true, all_preds)

            # TensorBoard Logging
            writer.add_scalar('Train Loss', train_loss, epoch + 1)
            writer.add_scalar('Validation Loss', val_loss, epoch + 1)
            writer.add_scalar('MAE', mae, epoch + 1)

            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, MAE: {mae:.4f}")

            # Early stopping
            # if early_stopping(val_loss):
            #     print("Early stopping triggered!")
            #     break

        val_losses.append(val_loss)
        mae_scores.append(mae)

    avg_val_loss = np.mean(val_losses)
    avg_mae = np.mean(mae_scores)
    print(f"Average Validation Loss: {avg_val_loss:.4f}, Average MAE: {avg_mae:.4f}")
    
    writer.close()  # Close TensorBoard writer


In [7]:
num_fold = 5
# Hyperparameter tuning
window_size = 48
learning_rates = 1e-4
batch_size = 64
val_window_size = len(dst_values) // (8 + 2 * num_fold) * 2  # Size of validation window
train_window_size = val_window_size * 4  # Size of training window

print(len(dst_values), window_size)

# for window_size in window_sizes:
#     for lr in learning_rates:
#         for batch_size in batch_sizes:
#             print(f"Training with window_size={window_size}, lr={lr}, batch_size={batch_size}")
#             train_with_sliding_window(dst_values, window_size, train_window_size, val_window_size, lr=lr, batch_size=batch_size, model_type="LSTM")


438841 48


In [ ]:
train_with_sliding_window(dst_values, window_size, train_window_size, val_window_size, lr=learning_rates, batch_size=batch_size, num_epochs=10)

OK1
OK2
OK3
OK4
Training fold 1/5...
OK5
OK6
OK7
OK8
OK9
1
